# Error analysis

First, we save in both a CSV and a XLSX file the predictions for every class: in this way, we can see exactly with how much certainty the ML model predicts a class over the other two.

In [ ]:
# ---- IMPORT LIBRARIES
from transformers import pipeline,AutoImageProcessor,AutoModelForImageClassification
import os
import numpy as np 
import pandas as pd
from tqdm import tqdm 
import pandas as pd



In [ ]:
# =========
# =============
# ================== VARIABLES
# =============
# =========

CHECKPOINT = './VALIDATION-baseC1-w_40e'

In [ ]:
""" 
from datasets import load_dataset

train_ds = load_dataset('imagefolder', data_dir='./dataset_binary/', split='train', streaming=True)
# ---- CHECKING CONVERSION FROM INTEGER VALUE TO LABEL NAME

labels = train_ds.features['label']

x = labels.int2str(0) # not sensitive
y = labels.int2str(1) # sensitive
# z = labels.int2str(2) # To be used only for a threeway classification 

print("Integer '0' is label '{x}'\nInteger '1' is label '{y}'".format(x=x, y=y)) # \nInteger '2' is label '{z}' + z=z """

In [ ]:
# ---- LOAD IMAGE PROCESSOR
finetuned_image_processor = AutoImageProcessor.from_pretrained(CHECKPOINT)


# ---- LOAD MODEL
model = AutoModelForImageClassification.from_pretrained(
    CHECKPOINT,
    label2id={"not-sensitive":0, "sensitive":1},
    id2label={0:"not-sensitive", 1:"sensitive"},
)


pipe = pipeline("image-classification", model=model, feature_extractor=finetuned_image_processor)
pipe


In [ ]:
# Take all images of a class 

not_sensitive_path = './dataset_binary/test/not-sensitive/'
sensitive_path = './dataset_binary/test/sensitive/'


def groupImages(path:str):
    img_lst = list()
    files = os.listdir(path)
    for file in files:
        f = os.path.join(path,file)
        img_lst.append(f)
    return img_lst

def restoreFileName(path:str):
    abs_path = os.path.abspath(path)
    file_name = os.path.basename(abs_path)
    return file_name
    

#dub_img_lst = groupImages(dubious_path)
ns_img_lst = groupImages(not_sensitive_path)
s_img_lst = groupImages(sensitive_path)

dataset_imgs = [(ns_img_lst,'not-sensitive'),(s_img_lst,'sensitive')]   #(dub_img_lst,'dubious'),




In [ ]:


def createArray(img_lst:list,annotation_value:str):
    # Create temporary lists with the various information
    lst1_imgnames = list()
    lst2_annotation = list()
    lst3_labels = list()
    lst4_scores = list()
    
    # Classes/labels names
    classes = ['not-sensitive','sensitive']

    # Scroll image list
    for image in img_lst:
        # Make prediction for that single image using the previously defined pipeline
        prediction = pipe(image)
        
        # Add the static information (image name, true annotation value, different classes)
        for i in range(3):
            lst1_imgnames.append(restoreFileName(image))
            lst2_annotation.append(annotation_value)
        for label in classes:
            lst3_labels.append(label)
            
        # Add the scores information: for each of the classes, what is the predicted score by the ML model?
            # These 3 scores need to be added in a precise order (corresponding to the classes order: dubious/0, not-sensitive/1, sensitive/2), but they are not always ordered in the pipeline output (list of dictionaries, one for each class)
            # First, sort the list of dictionaries by the value of key['label']: luckily, the alphabetical order of the three strings is the desired one (d/0-n/1-s/2)
        sorted_prediction = sorted(prediction, key=lambda d:d['label'])
        
        # Now we can append the scores values to the lst4_scores
        for dict in sorted_prediction:
            lst4_scores.append(dict['score'])
        
    # Populate array
    arr1_imgnames = np.array(lst1_imgnames)
    arr2_annotation = np.array(lst2_annotation)
    arr3_labels = np.array(lst3_labels)
    arrays = [arr1_imgnames,arr2_annotation,arr3_labels]
        
    return arrays, lst4_scores


""" for lst,ann in tqdm(dataset_imgs):
    arrays, lst4_scores = createArray(lst,ann)
    df = pd.DataFrame(lst4_scores,index=arrays,columns=['scores'])
    df.to_excel('VALIDATION-baseC1-w_40e', merge_cells = True)
 """

## Wrong predictions

However, we are interested specifically in all the images which have been predicted *wrongly*. We save the results in a pandas dataframe which then we export both as a CSV and as an Excel file.

In [ ]:

wrong_predictions = pd.DataFrame(columns=['image','annotation value','predicted label','score'])

for lst,ann in dataset_imgs:
    for img in lst:
        preds = pipe(img,top_k=1)
        for dict in preds:
            if dict['label'] != ann:
                wrong_predictions.loc[len(wrong_predictions.index)] = [restoreFileName(img), ann, dict['label'],dict['score']]  

wrong_predictions.to_excel('./valid_error_analysis.xlsx',index=False)
wrong_predictions.to_csv('./valid_error_analysis',index=False)    